In [8]:
import pandas as pd
import numpy as np

Combining Data
    
    - pandas 에서는 서로 다른 데이터 소스를 결합하기 위한 메서드들을 제공
        - concat() : 서로 다른 두 데이터를 단순 연결 np.concatenate()
        - merge(), join() : 다양한 데이터 조인, 병합 옵션 제공
    
    - Pandas의 concat() : np.concatenate() 보다 다양한 옵션 제공
        - 함수 시그니처(함수 원형)
        pandas.concat(objs, axis =0, join = 'outer', ignore_index = False, Keys = None, 
            levels = None, names = None, verify_integrity = False, sort = False, copy = True)
        - objs : 연결대상 객체들
        - axis : 연결 차원 축 axis = 0 ( 수직) / aixs = 1 (수평)
        - join : 조인 옵션
        - ignore_index : 기존의 인덱스 무시
    - concatenate() 보다 concat()이 더 편한 이유
        - 인덱스 중복 허용
        - 인덱스 중복 에러 처리: verify_integrity = True
        - 인덱스 무시 : ignore_index = True
        - 계층적 인덱싱 지원 : Keys = [Keys] (참고)
    - concat () 조인 옵션 : 서로 다른 열 집합을 가지는 객체를 연결 
        -> 따로 제공하지만 concat에서 join 옵션을 설정해줄 수 있음. / join = outer : 합 집합 join = inner : 교집합
    - pandas는 고성능 인메모리 조인과 병합 연산 기능을 제공
    - 조인 연산, join(): 인덱스를 기준으로 두 DataFrame을 결합
    - 병합 연산, merge() 데이터셋 조인/병합 관련 다양한 옵션 제공
    
    - Relational Algebra ( 관계 대수 )
        - 관계 데이터의 연산 구칙을 수학적으로 나타낸 이론
            - 관계형 데이터베이스(RDBMS)의 수학적 기반
        - 조인 연산 유형
            - 일대일(one-to-one)조인 : 공통 열의 중복 항목 없음
            - 다대일(many-to-one)조인 : 두 DataFrame의 키 중에 하나가 중복된 항목을 포함
            - 다대다(many-to-many)조인 : 두 DataFrame의 키가 모두 중복된 항목을 포함
            
        - merge() 에서 제공하는 데이터 결합 옵션들
            - on: 병합에 사용할 공통 키
            - left_on, right_on : 공통 키 역할을 하는 두 DataFrame 의 열
            - left_index, right_index : 인덱스를 이용한 병합 여부
            
        - merge() 조인 옵션
            - how = inner : 내부 조인(디폴트) -> 교집합
                - pd.concat(): 외부 조인이 디폴트
            - how = outer: 외부 조인 -> 합집합
            - how = left: 왼쪽 조인(첫 번째 DataFrame 기준으로 조인)
            - how = right: 오른쪽 조인(두 번째, DataFrame 기준으로 조인)

In [50]:
"""
    numpy.concatenate()
"""
x = [1,2,3]
y = [4,5,6]
z = [7,8,9]
print(np.concatenate([x,y,z]))
# vstack() : concatenate vertically
# hstack() : concatenate horizontally

print(np.vstack([x,y,z]))

[1 2 3 4 5 6 7 8 9]
[[1 2 3]
 [4 5 6]
 [7 8 9]]


In [51]:
"""
pandas.concat()
"""
ser1 = pd.Series(['A','B','C'], index = [1,2,3])
ser2 = pd.Series(['D','E','F'], index = [4,5,6])

print(pd.concat([ser1,ser2])) # 명시적 index이기 때문에 4,5,6,1,2,3 순으로 되도 상관 없다.

# Simply concatenate the two Series objects

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object


In [52]:
# make_df(): create a DataFrame object
def make_df(cols, ind):
    data = {c : [str(c) + str(i) for i in ind]
           for c in cols}
    
    return pd.DataFrame(data,ind)

make_df('ABC', [1,2]) # 2 by 3

,A,B,C
1,A1,B1,C1
2,A2,B2,C2


In [53]:
# concatenate the two DataFrame objects
df1 = make_df('AB',[1,2])
df2 = make_df('AB', [3,4])
print(df1, '\n')
print(df2, '\n')

print(pd.concat([df1,df2]))

    A   B
1  A1  B1
2  A2  B2 

    A   B
3  A3  B3
4  A4  B4 

    A   B
1  A1  B1
2  A2  B2
3  A3  B3
4  A4  B4


In [54]:
df3 = make_df('AB',[0,1])
df4 = make_df('CD',[0,1])

print(df3, '\n')
print(df4, '\n')

print(pd.concat([df3,df4],axis = 1), '\n')
print(pd.concat([df3,df4],axis = 0), '\n')

    A   B
0  A0  B0
1  A1  B1 

    C   D
0  C0  D0
1  C1  D1 

    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1 

     A    B    C    D
0   A0   B0  NaN  NaN
1   A1   B1  NaN  NaN
0  NaN  NaN   C0   D0
1  NaN  NaN   C1   D1 



In [55]:
"""
    concate() Features
"""

# case1: Duplicate indices
x = make_df('AB',[0,1])
y = make_df('AB',[2,3])
print(y)
y.index = x.index
print(y)

print(pd.concat([x,y]))

    A   B
2  A2  B2
3  A3  B3
    A   B
0  A2  B2
1  A3  B3
    A   B
0  A0  B0
1  A1  B1
0  A2  B2
1  A3  B3


In [56]:
# case 2 : Catching duplications as erros
try:
    pd.concat([x,y], verify_integrity = True)
except ValueError as e:
    print("ValueError: ", e)

ValueError:  Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


In [57]:
# case 3 : Ignore the Index:

print(pd.concat([x,y], ignore_index = True)) # 인덱스를 다 날리고 zero base 정수로 자동으로 재할당해줌.


    A   B
0  A0  B0
1  A1  B1
2  A2  B2
3  A3  B3


In [58]:
# case 4: Adding multiindex Keys (hirechical)

In [59]:
# default : outer join -> union
print(x)
print(y)
print(pd.concat([x,y], join = 'outer'), '\n')
print(pd.concat([x,y], join = 'inner'))

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
    A   B
0  A0  B0
1  A1  B1
0  A2  B2
1  A3  B3 

    A   B
0  A0  B0
1  A1  B1
0  A2  B2
1  A3  B3


In [60]:
"""
join operator
"""

'\njoin operator\n'

In [61]:
# one-to-one

In [62]:
# many-to-one

In [63]:
# many-to-many

In [70]:
# 'on' argument: use a common column as merge key

df1['age'] = pd.Series([20,42,37,25])
df2['age'] = pd.Series([37,20,42,25])
print(df1, '\n')
print(df2, '\n')

# print(pd.merge(df1, df2, on = 'employee')) # -> age 기준으로 merge

    A   B  age
1  A1  B1   42
2  A2  B2   37 

    A   B   age
3  A3  B3  25.0
4  A4  B4   NaN 



In [71]:
# 'left_on', 'right_on' argument
# pd.merge(df1, df3, left_on = 'employee', right_on = 'name')

In [74]:
print(df1, '\n')
print(df2, '\n')

df1a = df1.set_index('employee')
df2a = df1.set_index('employee')
print(df1a, '\n')
print(df2a, '\n')
# print(pd.merge(df1a, df2a, left_index = True, right_index = True))

    A   B  age
1  A1  B1   42
2  A2  B2   37 

    A   B   age
3  A3  B3  25.0
4  A4  B4   NaN 



KeyError: "None of ['employee'] are in the columns"

In [ ]:
"""
    Specifying the join method
"""